In [21]:
import numpy as np
import numpy.typing as npt

NPVector = npt.NDArray[np.float64]

In [22]:
def sigmoid(z: NPVector) -> NPVector:
    return 1 / (1 + np.exp(-z))


def softmax(z: NPVector) -> NPVector:
    z = np.array(z)
    exp_z = np.exp(z - np.max(z))
    return exp_z / np.sum(exp_z)


def relu(z: NPVector) -> NPVector:
    return np.maximum(0, z)

In [23]:
class Layer:
    def __init__(self, input_dim: int, units: int, activation):
        self.activation = activation
        self.W = np.random.rand(units, input_dim) * 0.1
        self.b = np.zeros((units, 1))

    def forward(self, x: NPVector) -> NPVector:
        self.x = x
        self.z = self.W @ x + self.b
        self.a = self.activation(self.z)

        return self.a

    def backward(self):
        ...


In [24]:
class NeuralNetwork:
    def __init__(self, layers: list[Layer]):
        self.layers = layers

    def forward(self, x: NPVector) -> NPVector:
        for layer in self.layers:
            x = layer.forward(x)

        return x

    def backward(self):
        ...

    def train(self):
        ...

    def predict(self):
        ...